In [10]:
import numpy as np
import sys

class RSMFeasibleOptimal:

    def __init__(self, A, b, c, x, y, bv, E=[]):
       
        # Initialize input parameters
        self.A = np.array(A, dtype='float').T
        self.b = np.array(b, dtype='float')
        self.c = np.array(c, dtype='float')
        self.x = np.array(x, dtype='float')
        self.y = np.array(y, dtype='float')
        self.bv = np.array(bv)
        self.nbv = np.array([z for z in range(len(self.x)) if z not in self.bv])
        self.m = self.A.shape[1]
        self.obj = 0
        self.exit_index = 0
        self.entry_index = 0

        # Initialize Eta matrices
        if E == []:
            self.E = []
            self.E.append(np.eye(self.m + 1))
        else:
            self.E = E

        self.e_index = 0
        self.pbar = np.zeros(self.m)

    def cal_coeff(self, index):
       
        return self.c[index] - np.dot(self.y, np.transpose(self.A[index]))

    def p_solve(self, e, p):
       
        pt = np.zeros(len(p))
        pt[e[-1]] = p[e[-1]] / e[e[-1]]
        for i in [x for x in range(len(p)) if x != e[-1]]:
            pt[i] = p[i] - e[i] * pt[e[-1]]
        return pt

    def cal_pbar(self, p):
       
        pt = p
        for ei in self.E:
            pt = self.p_solve(ei, pt)
        return pt

    def y_solve(self, e, cb):
        
    
        yt = cb
        s = 0
        for i in [x for x in range(len(cb)) if x not in e[:-1]]:
            s = s + yt[i] * e[i]
        yt[e[-1]] = (cb[e[-1]] - s) / e[e[-1]]
        return yt


    def cal_y(self, cb):
      
        yt = cb
        for e in reversed(self.E):
            yt = self.y_solve(e, yt)
        return yt

    def cal_obj(self):
       
        return np.dot(self.x, self.c)

    def solve(self):
       
        while True:
            cb = [self.c[i] for i in self.bv]
            self.y = self.cal_y(cb)
            self.obj = self.cal_obj()
            maximum = -1

            # Find entering variable
            for i in self.nbv:
                coeff = self.cal_coeff(i)
                if (coeff > maximum or (coeff == maximum and i < self.entry_index)) and coeff > 0:
                    maximum = coeff
                    self.entry_index = i

            # Check optimality
            if maximum == -1:
                return "Optimal"

            self.pbar = self.cal_pbar(self.A[self.entry_index])

            # Check unboundedness
            if all(i <= 0 for i in self.pbar):
                return "Unbounded"

            theta = sys.maxsize
            flag = 0

            # Find the exiting variable
            for i in range(self.m):
                th = self.x[self.bv[i]] / self.pbar[i]
                if th > 0 and ((th < theta) or (th == theta and self.bv[i] < self.exit_index)):
                    theta = th
                    self.e_index = i
                    self.exit_index = self.bv[i]
                if th == 0:
                    for i in range(self.m):
                        if self.pbar[i] > 0:
                            flag = 1
                            break
                    if flag == 1:
                        theta = 0
                        self.exit_index = self.bv[i]
                        self.e_index = i
                        break

            self.x[self.entry_index] = theta
            for i in range(self.m):
                self.x[self.bv[i]] = self.x[self.bv[i]] - self.pbar[i] * theta
            self.x[self.exit_index] = 0.0

            e = np.zeros(len(self.pbar) + 1)
            e[-1] = self.e_index
            for i in range(len(self.pbar)):
                e[i] = self.pbar[i]
            self.E.append(e)
            self.bv[self.bv == self.exit_index] = self.entry_index
            self.nbv[self.nbv == self.entry_index] = self.exit_index

# Sample Case
A = [[2, 1], [1, 1], [1, 0]]
b = [10, 8, 3]
c = [3, 2]
x = [0, 0]
y = [0, 0]
bv = [0, 1]

rsm = RSMFeasibleOptimal(A, b, c, x, y, bv)
result = rsm.solve()
print("Result:", result)
print("Optimal Solution (x):", rsm.x)
print("Optimal Objective Value:", rsm.cal_obj())

TypeError: only integer scalar arrays can be converted to a scalar index